### **Data Cleaning Rules Applied**

---

### 1.  **Dropping Irrelevant and Sparse Features**

**Issue:**
Some columns were either irrelevant to insurance pricing or contained sparse data that could not be meaningfully imputed.

**Solution:**
The following columns were dropped:

* `'Unique number'`, `'Citizenship'`, `'Gender'`: Not predictive of insurance price.
* `'Loss_amount'`, `'Accident_region'`: Sparse features with many missing or inconsistent values.

```python
df.drop(columns=['Unique number', 'Citizenship', 'Gender', 'Loss_amount', 'Accident_region'], inplace=True)
```

---

### 2. **Fixing Driving Experience**

**Issue:**
In some rows, driving experience exceeded the driver’s age or indicated that the person started driving before age 18, which is illegal in Kazakhstan.

**Solution:**

* Created a helper column `Age_gap = Age - Driving_experience`.
* Dropped rows where `Driving_experience > Age`.
* Adjusted values where driving started before age 18 by reducing the reported experience.
* Dropped the temporary `Age_gap` column after corrections.

---

### 3. **Column Name Cleanup**

**Issue:**
Some column names contained trailing spaces, which can cause errors during access or processing.

**Solution:**
All column names were stripped of leading/trailing whitespace:

```python
df.columns = df.columns.str.strip()
```

---

### 4. **Vehicle Type Filtering and Mapping**

**Issue:**
The `Vehicle_type` column included trailer types and inconsistent naming for vehicle categories.

**Solution:**

* Removed trailer-related rows:

  * `'Прицеп к грузовой а/м'`
  * `'Прицеп к легковой а/м'`

* Reduced number of vehicle types by grouping into broader standardized categories:

  ```text
  'Легковые автомобили' → 'Легковые автомобили'
  'Мотоциклы и мотороллеры' → 'Мотоциклы'
  'Грузовые автомобили' → 'Грузовые'
  'Автобусы до 16 п/м вкл.' and 'Автобусы, свыше 16 п/м' → 'Автобусы'
  ```

---

### 5. **Car Age Calculation**

**Issue:**
The `Year_of_manufacture` feature needed to be converted into a usable numerical format.

**Solution:**

* Parsed the year into datetime format.
* Computed a new feature `Car_age = 2025 - manufacture_year`.
* Dropped the original `'Year_of_manufacture'` column after calculation.

---

### 6.**Insurance Period Parsing and Duration Calculation**

**Issue:**
The `Insurance_period` column stored date ranges as a single string, preventing direct use in modeling.

**Solution:**

* Split `Insurance_period` into `start_date` and `end_date`.
* Converted both to datetime format.
* Calculated insurance duration in months using `relativedelta`.
* Dropped the intermediate columns.
* Only the new feature is left: `'Insurance_months'`.

---

### 7.**Filling Missing Privileges**

**Issue:**
The `Privileges` column had missing values, which could affect model performance or interpretations.

**Solution:**
All null entries were filled with `'Не инвалид'`, assuming the absence of a value indicates no disability benefit.

```python
df['Privileges'] = df['Privileges'].fillna('Не инвалид')
```


---

#### 8. **Color Mapping**

**Issue:**
The dataset contained **177 unique color names**, many of which were variants or synonyms (e.g., spelling differences, shades, and compound names). This level of granularity introduced noise and complexity into the feature.

**Solution:**

* A predefined `color_mapping` dictionary was used to **consolidate color names into 9 main color groups** such as `'Белый'`, `'Черный'`, `'Синий'`, etc.
* For example:

  * `'черный металлик'` and `'черный с фиолетовым отливом'` were both mapped to `'Черный'`.
  * `'жемчужно-белый'`, `'снежная королева'`, and similar names were mapped to `'Белый'`.

**Handling Rare or Unmapped Values:**

* Any color not found in the dictionary was assigned to a **catch-all group** labeled `'Прочие'` (meaning "Other").

```python
df['Color'] = df['Color'].apply(lambda col: color_mapping.get(col.strip().lower(), 'Прочие'))
```

---

#### 9. **Standardizing Car Brands and Models**



*1. Brand Name Normalization*

**Issue:**
The brand "Lada" appeared in two formats:

* Latin script: `"Lada"`
* Cyrillic: `"Лада"`

**Solution:**
To ensure consistency, all instances of `'Лада'` were **replaced with `'Lada'`**:



*2. Grouping Rare Brands*

**Observation:**
The dataset contained many brands with very few entries, which could introduce noise.

**Solution:**
The top **37 most frequent brands** were retained. All others were grouped under the label `'Other'`:

```python
top_brands = df['Brand'].value_counts().nlargest(37).index
df['Brand'] = df['Brand'].apply(lambda b: b if b in top_brands else 'Other')
```



*3. Cleaning and Grouping Car Models*

**Issue:**

* There were **1,101 unique car model names**, including many highly specific or uncommon values.
* Some entries had a placeholder value of `"."`, which lacks semantic meaning and could not be reliably imputed due to sparse data.

**Solution:**

* Replaced `"."` values with `'Unknown'`:

  ```python
  df['Model'] = df['Model'].replace('.', 'Unknown')
  ```

* Retained only the **top 50 most frequent model names** as separate groups.

* Grouped all other model names under `'Other'`:

  ```python
  top_models = df['Model'].value_counts().head(50).index
  df['Model'] = df['Model'].apply(lambda m: m if m in top_models else 'Other')
  ```

---


### 10. **City Cleaning and Region Mapping**

**Issue:**

* The `City` column contained inconsistent entries, including **duplicate or compound names** (e.g., `"Костанай, Костанай"`).
* The `Accident_region` column was highly **sparse**, with many missing values.
* Having two overlapping location fields introduced redundancy and inconsistency.

**Solution:**

* A new column `City_cleaned` was created by extracting only the first part of each city name:

  ```python
  df['City_cleaned'] = df['City'].astype(str).apply(lambda x: x.split(',')[0].strip())
  ```

* A custom `region_mapping` dictionary was defined to map each cleaned city to its corresponding **region** (область).

* The new feature `Region` was created by applying this mapping:

  ```python
  df['Region'] = df['City_cleaned'].map(region_mapping)
  ```

* The original columns `City` and `City_cleaned` were dropped:

  ```python
  df.drop(columns=['City', 'City_cleaned'], inplace=True)
  ```

* Missing region values were filled with `'Unknown'`:

  ```python
  df['Region'] = df['Region'].fillna('Unknown')
  ```

---




In [ ]:
import pandas as pd
from dateutil.relativedelta import relativedelta

pd.reset_option('display.max_rows')

In [29]:
df = pd.read_excel("./data/raw_data.xlsx")
df.sample(5) 

,Unique number,Age,Citizenship,Driving_experience,Gender,Bonus_Malus,City,Model,Brand,Vehicle_type,Year_of_manufacture,Steering_wheel_location,Color,Insurance_period,Insurance_premium,Privileges,Accident_region,Loss_amount
9004,d83bbdead462b48b283173a667c66e38,43,Казахстан,3,М,7,Алматы,Polo,Volkswagen,Легковые автомобили,2007-12-01,Слева,синий,29.06.2013-05.01.2014,8518,NaN,NaN,NaN
4941,f7e0d30324c8058a3b652c4eb9a4f818,64,Казахстан,20,М,9,Алматы,Patrol,Nissan,Легковые автомобили,2003-06-01,Слева,бордовый,02.07.2013-01.07.2014,7833,Инвалид,NaN,NaN
926,ce2aa9774e1319f57d37e6571e6bf774,69,Казахстан,8,М,7,Алматы,X5,BMW,Легковые автомобили,2001-01-01,Слева,синий металлик,04.06.2013-03.12.2013,9538,NaN,NaN,NaN
3360,8fd61f1f0a0edba36234d1f2914648a8,44,Казахстан,19,М,9,Алматы,Sprinter 316 Cdi,Mercedes-Benz,Легковые автомобили,2009-12-01,Слева,синий,15.06.2013-14.06.2014,14243,NaN,NaN,NaN
10575,9d7694837e4d2a5445eafcda4d76b8fa,66,Казахстан,43,М,8,Костанай,.,КАМАЗ,Грузовые автомобили,1991-01-01,Слева,зелёный,23.05.2013-22.11.2013,10616,NaN,NaN,NaN


In [30]:
df.shape

(10909, 18)

In [31]:
for col in df.columns:
    print(f"--- Value counts for: {col} ---")
    print(df[col].value_counts(dropna=True))
    print("\n")

--- Value counts for: Unique number ---
Unique number
b7c927b072a0912dbdc401d48ad68134    20
ddece7b2c11c98067d2afb2c5f9b76a4    18
d0007134147a341a79b8983b1c2203cd    18
1afa374ad63fc594e2bd2310e4c93430    15
29c7cbd8a3f4ab746be4428b68718532    14
                                    ..
fe9137718c962a94db3910a09a65e527     1
003302549bec6608b65f3775f810b95b     1
073276060e764ae0dfea7cae0cf13219     1
6c7c2ad824f7f7acc01bcf1f14ede472     1
02d81ab4c537df1fc7e8a5a128640702     1
Name: count, Length: 9420, dtype: int64


--- Value counts for: Age ---
Age
41    342
37    331
38    319
46    310
47    309
     ... 
86      2
89      1
87      1
90      1
91      1
Name: count, Length: 68, dtype: int64


--- Value counts for: Citizenship ---
Citizenship
Казахстан      10866
Россия            37
Кыргызстан         2
Таджикистан        1
Индия              1
Молдова            1
Украина            1
Name: count, dtype: int64


--- Value counts for: Driving_experience ---
Driving_experience
19

In [32]:
df.isna().sum()

Unique number                  0
Age                            0
Citizenship                    0
Driving_experience             0
Gender                         0
Bonus_Malus                    0
City                           0
Model                          0
Brand                          0
Vehicle_type                   0
Year_of_manufacture            0
Steering_wheel_location        0
Color                          0
Insurance_period               0
Insurance_premium              0
Privileges                  9713
Accident_region            10773
Loss_amount                10774
dtype: int64

In [33]:
df.drop(columns=['Unique number', 'Citizenship', 'Gender', 'Loss_amount', 'Accident_region'], inplace=True)

In [34]:
df[df['Age'] < 18]

,Age,Driving_experience,Bonus_Malus,City,Model,Brand,Vehicle_type,Year_of_manufacture,Steering_wheel_location,Color,Insurance_period,Insurance_premium,Privileges


In [35]:
df['Age_gap'] = df['Age'] - df['Driving_experience']
df['Age_gap'].describe()

count    10909.000000
mean        29.335869
std         11.148937
min        -29.000000
25%         20.000000
50%         26.000000
75%         36.000000
max         77.000000
Name: Age_gap, dtype: float64

The 'Age_gap' is the difference between the age of the driver and driver experience. The negative min value of Age_gap indicates that there is inconsistent value, where the age of the driver is less than the driving experience.

In [36]:
invalid_experience = df[df['Driving_experience'] > df['Age']]
invalid_experience

,Age,Driving_experience,Bonus_Malus,City,Model,Brand,Vehicle_type,Year_of_manufacture,Steering_wheel_location,Color,Insurance_period,Insurance_premium,Privileges,Age_gap
2775,59,88,7,Караганда,626,Mazda,Легковые автомобили,1995-12-01,Слева,темно-зеленый,04.06.2013-03.06.2014,8408,NaN,-29


This observation was dropped.

In [37]:
df.drop(index=invalid_experience.index, inplace=True)

According to Kazakhstan's driving rules, a citizen may acquire driving license after 18 years old. There are observations that indicate drivers who started driving under 18 years old.

In [38]:
started_under_18 = df[(df['Age_gap'] < 18)]
started_under_18

,Age,Driving_experience,Bonus_Malus,City,Model,Brand,Vehicle_type,Year_of_manufacture,Steering_wheel_location,Color,Insurance_period,Insurance_premium,Privileges,Age_gap
10,32,15,8,"Атбасар,Атбасар",80,Audi,Легковые автомобили,1988-01-01,Слева,красный,13.06.2013-12.06.2014,5988,NaN,17
47,41,24,9,Костанай,100,Audi,Легковые автомобили,1992-12-01,Слева,темно-синий металлик,18.06.2013-17.06.2014,10321,NaN,17
62,59,42,9,Актобе,80,Audi,Легковые автомобили,1988-12-01,Слева,серебристый,20.06.2013-19.06.2014,7145,NaN,17
79,56,39,8,Есиль,С4,Audi,Легковые автомобили,1997-12-01,Слева,синий,28.06.2013-27.06.2014,5988,NaN,17
86,37,20,3,Горняцкий,B4,Audi,Легковые автомобили,1992-12-01,Слева,зеленый металлик,03.07.2013-02.07.2014,12385,NaN,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,38,21,7,Атбасар,М-2140,Москвич,Легковые автомобили,1980-01-01,Слева,голубой,14.06.2013-13.12.2013,3203,NaN,17
10823,25,8,4,"Красный Яр,Красный Яр",3303,УАЗ,Легковые автомобили,1988-12-01,Слева,желтый,09.06.2013-08.12.2013,4992,NaN,17
10851,58,42,8,Актау,3163 235,УАЗ,Легковые автомобили,2012-12-01,Слева,серебристый металлик архивный,05.06.2013-04.06.2014,5929,Инвалид,16
10882,44,27,8,Крамское,31512,УАЗ,Легковые автомобили,1991-01-01,Слева,песочный,08.06.2013-07.06.2014,11058,NaN,17


These observations' values were rounded as if drivers started driving at 18 years old.

In [39]:
adjustment = 18 - started_under_18['Age_gap']

df.loc[started_under_18.index, 'Driving_experience'] -= adjustment

df[df['Driving_experience'] < 0]

,Age,Driving_experience,Bonus_Malus,City,Model,Brand,Vehicle_type,Year_of_manufacture,Steering_wheel_location,Color,Insurance_period,Insurance_premium,Privileges,Age_gap


In [40]:
df.drop(columns=['Age_gap'], inplace=True)

In [41]:
print(df.columns.tolist()) # there is a space in the name of the column 'Vehicle_type '
df.columns = df.columns.str.strip() # remove all the spaces in the name of columns
df['Vehicle_type'].value_counts()

['Age', 'Driving_experience', 'Bonus_Malus', 'City', 'Model', 'Brand', 'Vehicle_type ', 'Year_of_manufacture', 'Steering_wheel_location', 'Color', 'Insurance_period', 'Insurance_premium', 'Privileges']


Vehicle_type
Легковые автомобили        10515
Грузовые автомобили          264
Автобусы до 16 п/м вкл.       50
Мотоциклы и мотороллеры       47
Автобусы, свыше 16 п/м        26
Прицеп к грузовой а/м          4
Прицеп к легковой а/м          2
Name: count, dtype: int64

'Прицеп к грузовой а/м', ''Прицеп к легковой а/м' were dropped due to small observations count.

'Автобусы до 16 п/м вкл.' and 'Автобусы, свыше 16 п/м' were united into one group 'Автобусы'.

There are 4 types of 'Vehicle_type'.

In [42]:
df = df[~df['Vehicle_type'].isin([
    'Прицеп к грузовой а/м',
    'Прицеп к легковой а/м'
])]

vehicle_mapping = {
    'Легковые автомобили': 'Легковые автомобили',
    'Мотоциклы и мотороллеры': 'Мотоциклы',
    'Грузовые автомобили': 'Грузовые',
    'Автобусы до 16 п/м вкл.': 'Автобусы',
    'Автобусы, свыше 16 п/м': 'Автобусы'
}


df.loc[:, 'Vehicle_type'] = df['Vehicle_type'].map(vehicle_mapping)

print(df['Vehicle_type'].value_counts())

Vehicle_type
Легковые автомобили    10515
Грузовые                 264
Автобусы                  76
Мотоциклы                 47
Name: count, dtype: int64


'Year_of_manufacture' solumn is in the format of 'yyyy-mm-dd'. This column was transformed into 'Car_age'.

In [43]:
df['Year_of_manufacture'] = pd.to_datetime(df['Year_of_manufacture'])
df['Car_age'] = 2025 - df['Year_of_manufacture'].dt.year
df.drop(columns=['Year_of_manufacture'], inplace=True)

'Insurance_period' column was in the format 'dd.mm.yyyy-dd.mm.yyyy'. The column was transformed into 'Insurance_months' indicating number of months of the insurance.

In [44]:
df[['start_date', 'end_date']] = df['Insurance_period'].str.split('-', expand=True)

df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')
df['end_date'] = pd.to_datetime(df['end_date'], format='%d.%m.%Y')


def calculate_months(row):
    diff = relativedelta(row['end_date'], row['start_date'])
    return diff.years * 12 + (diff.months + 1)

df['Insurance_months'] = df.apply(calculate_months, axis=1)
df.drop(columns=['Insurance_period', 'start_date', 'end_date'], inplace=True)

In [45]:
df['Insurance_months'].value_counts()

Insurance_months
12    8173
6     1619
7      244
8      200
10     176
11     166
9      166
1      147
3        7
5        2
2        2
Name: count, dtype: int64

About 89% of'Privileges' column's values are NaN. They were assumed as 'Не инвалид'.

In [46]:
print("Number of NaN values in 'Privileges' column: ", df['Privileges'].isna().sum())
df['Privileges'].value_counts()

Number of NaN values in 'Privileges' column:  9707


Privileges
Инвалид                  1035
Пенсионер                  82
Приравн.Лицо               61
Инвалид Пенсионер          11
Участник ВОВ                4
Приравн.Лицо Инвалид        2
Name: count, dtype: int64

In [47]:
df['Privileges'] = df['Privileges'].fillna('Не инвалид')

There are 177 unique color names. They were groupped into 10 categories. One of them is 'Прочие' which gathers color names that were not classified into any other 9 groups.

In [48]:
print("Number of color names: ", df['Color'].nunique())
print("Number of color names with the count less than 100: ",(df['Color'].value_counts() < 100).sum())


color_mapping = {
    'белый': 'Белый', 'снежная королева': 'Белый', 'жемчужно-белый': 'Белый', 'бело-серый': 'Белый',
    'черный': 'Черный', 'черный металлик': 'Черный', 'черный с фиолетовым отливом': 'Черный',
    'серый': 'Серый', 'темно-серый': 'Серый', 'графит': 'Серый', 'мокрый асфальт': 'Серый',
    'синий': 'Синий', 'темно-синий': 'Синий', 'ярко-синий': 'Синий', 'сине-зеленый': 'Синий',
    'зеленый': 'Зеленый', 'лайм': 'Зеленый', 'изумрудный': 'Зеленый', 'оливковый': 'Зеленый',
    'красный': 'Красный', 'бордовый': 'Красный', 'вишневый': 'Красный', 'коралл': 'Красный',
    'желтый': 'Желтый', 'лимонный': 'Желтый', 'светло-желтый': 'Желтый',
    'коричневый': 'Коричневый', 'мокко': 'Коричневый', 'шоколадный': 'Коричневый',
    'фиолетовый': 'Фиолетовый', 'сиреневый': 'Фиолетовый', 'лиловый': 'Фиолетовый',
    # 'Прочие'
}
 # everything else will be mapped as 'Прочие'
df['Color'] = df['Color'].apply(lambda col: color_mapping.get(col.strip().lower(), 'Прочие'))

Number of color names:  177
Number of color names with the count less than 100:  155


The brand name "Lada" appeared in both Latin ("Lada") and Cyrillic ("Лада") formats. To ensure consistency, all Cyrillic entries were standardized to "Lada".

In addition, only the top 37 most frequent brands were kept as individual categories. All other less common brands were grouped together under the label "Other" to reduce noise and simplify the feature.

In [49]:
df['Brand'] = df['Brand'].replace({'Лада': 'Lada'})

top_brands = df['Brand'].value_counts().nlargest(37).index

df['Brand'] = df['Brand'].apply(lambda b: b if b in top_brands else 'Other')


There are 1101 unique car model names.

In [50]:
print("Number of models of the vechicle: ", df["Model"].nunique())
models = df['Model'].value_counts()
models

Number of models of the vechicle:  1101


Model
Camry       437
Passat      371
.           325
Golf        232
Rav 4       229
           ... 
Wagon r+      1
Jimny         1
Bandit        1
80 Avant      1
ИМЗ 8103      1
Name: count, Length: 1101, dtype: int64

The Model column contained a placeholder value "." to represent missing or unknown models. These unknown entries were very scattered, with no dominant pattern or frequent combination across Brand, Car_age, and Vehicle_type. This made it difficult to reliably impute or infer the correct model.

In [51]:
df[df['Model'] == '.'].groupby(['Brand', 'Car_age', 'Vehicle_type']).size()

Brand  Car_age  Vehicle_type       
Audi   14       Легковые автомобили    1
       29       Легковые автомобили    1
       35       Легковые автомобили    1
BMW    23       Легковые автомобили    2
       27       Легковые автомобили    2
                                      ..
Иж     26       Легковые автомобили    1
       35       Легковые автомобили    1
КАМАЗ  13       Грузовые               2
       34       Грузовые               1
УАЗ    12       Легковые автомобили    1
Length: 183, dtype: int64

There is not enough density to impute the model.

In [52]:
df_unknown = df[df['Model'] == '.']
top_combinations = df_unknown.groupby(['Brand', 'Car_age', 'Vehicle_type']).size().reset_index(name='count')
top_combinations.sort_values('count', ascending=False).head(10)

,Brand,Car_age,Vehicle_type,count
79,Mercedes-Benz,34,Легковые автомобили,8
169,Volkswagen,33,Легковые автомобили,7
30,Daimler-Benz,37,Легковые автомобили,6
144,Renault,31,Легковые автомобили,6
77,Mercedes-Benz,32,Легковые автомобили,5
28,Daimler-Benz,27,Легковые автомобили,5
16,Daimler Chrysler,20,Легковые автомобили,5
109,Other,15,Легковые автомобили,5
76,Mercedes-Benz,31,Легковые автомобили,5
106,Other,13,Грузовые,5


The observations with the '.' model name were replaced with name 'Unknown'.

In [53]:
df['Model'] = df['Model'].replace('.', 'Unknown')

The top 50 models were defined as separate groups, the left models were grouped under the name 'Other'.

In [54]:
top_models = df['Model'].value_counts().head(50).index
df['Model'] = df['Model'].apply(lambda m: m if m in top_models else 'Other')

In [55]:
df.groupby('Model')['Insurance_premium'].agg(['mean', 'count']).sort_values('count', ascending=False)

,mean,count
Model,,
Other,10851.100018,5559
Camry,12293.439359,437
Passat,8322.450135,371
Unknown,9749.587692,325
Golf,10546.525862,232
Rav 4,13768.030568,229
Land Cruiser Prado,14096.295455,220
Pajero,12887.045000,200
80,7814.158537,164


Now our dataset looks like this

In [56]:
df.sample(5)

,Age,Driving_experience,Bonus_Malus,City,Model,Brand,Vehicle_type,Steering_wheel_location,Color,Insurance_premium,Privileges,Car_age,Insurance_months
10265,35,17,4,Алматы,Other,ГАЗ,Легковые автомобили,Слева,Белый,21262,Не инвалид,39,12
6676,41,23,8,Нур-Султан,Highlander,Toyota,Легковые автомобили,Слева,Прочие,6255,Не инвалид,22,6
9223,55,22,9,Алматы,Other,Volvo,Легковые автомобили,Слева,Прочие,15667,Не инвалид,30,12
9268,58,19,4,Кокшетау,Other,ВАЗ,Легковые автомобили,Слева,Красный,4754,Не инвалид,40,6
346,32,8,5,Нур-Султан,Other,Audi,Легковые автомобили,Слева,Синий,14971,Не инвалид,29,12


In [57]:
df.isnull().sum()

Age                        0
Driving_experience         0
Bonus_Malus                0
City                       0
Model                      0
Brand                      0
Vehicle_type               0
Steering_wheel_location    0
Color                      0
Insurance_premium          0
Privileges                 0
Car_age                    0
Insurance_months           0
dtype: int64

In [58]:
df.describe()

,Age,Driving_experience,Bonus_Malus,Insurance_premium,Car_age,Insurance_months
count,10902.000000,10902.000000,10902.00000,10902.000000,10902.000000,10902.000000
mean,47.351862,17.956797,7.36718,10933.732618,25.777472,10.673271
std,12.795866,8.712328,1.70596,5732.619701,7.735468,2.523356
min,24.000000,0.000000,1.00000,66.000000,12.000000,1.000000
25%,37.000000,12.000000,7.00000,6412.750000,19.000000,11.000000
50%,46.000000,18.000000,8.00000,9479.500000,26.000000,12.000000
75%,56.000000,20.000000,9.00000,15667.000000,31.000000,12.000000
max,91.000000,64.000000,9.00000,59669.000000,70.000000,12.000000


There were repeating city names in some observations of the 'City' column.

The 'City' and 'Accident_region' columns were dropped. The 'Accident_region' has sufficient amount of NaN values. 'Region' column was created instead, with cities grouped by region.

In [59]:
df['City_cleaned'] = df['City'].astype(str).apply(lambda x: x.split(',')[0].strip())


region_mapping = {
    'Алматы': 'Алматинская область',
    'Нур-Султан': 'Астана',
    'Актобе': 'Актюбинская область',
    'Петропавловск': 'Северо-Казахстанская область',
    'Кокшетау': 'Акмолинская область',
    'Костанай': 'Костанайская область',
    'Павлодар': 'Павлодарская область',
    'Караганда': 'Карагандинская область',
    'Семей': 'Восточно-Казахстанская область',
    'Актау': 'Мангистауская область',
    'Атырау': 'Атырауская область',
    'Уральск': 'Западно-Казахстанская область',
    'Талдыкорган': 'Алматинская область',
    'Шымкент': 'Туркестанская область',
    'Кызылорда': 'Кызылординская область',
    'Тараз': 'Жамбылская область',
    'Усть-Каменогорск': 'Восточно-Казахстанская область',
    'Рудный': 'Костанайская область',
    'Темиртау': 'Карагандинская область',
    'Есик': 'Алматинская область',
    'Атбасар': 'Акмолинская область',
    'Жаксы': 'Акмолинская область',
    'Есиль': 'Северо-Казахстанская область',
    'Красный Яр': 'Актюбинская область',
    'Мариновка': 'Костанайская область',
    'Запорожье': 'Костанайская область',
    'Новоалександровка': 'Костанайская область',
    'Балкашино': 'Акмолинская область',
    'Лозовое': 'Костанайская область',
    'Талгар': 'Алматинская область',
    'Есенгельды': 'Алматинская область',
    'Новокиенка': 'Костанайская область',
    'Борисовка': 'Северо-Казахстанская область',
    'Аршалы': 'Акмолинская область',
    'Максимовка': 'Акмолинская область',
    'Боралдай': 'Алматинская область',
    'Покровка': 'Актюбинская область',
    'Октябрьское': 'Актюбинская область',
    'Садовое': 'Северо-Казахстанская область',
    'Тимашевка': 'Костанайская область'
}

df['Region'] = df['City_cleaned'].map(region_mapping)



df['City_cleaned'] = df['City'].astype(str).apply(lambda x: x.split(',')[0].strip())

The Region column’s missing values were filled with 'Unknown'.

In [60]:
df.drop(columns=["City_cleaned", "City"], inplace=True)

df['Region'] = df['Region'].fillna('Unknown')

In [65]:
df.sample(5)

,Age,Driving_experience,Bonus_Malus,Model,Brand,Vehicle_type,Steering_wheel_location,Color,Insurance_premium,Privileges,Car_age,Insurance_months,Region
2717,35,14,7,323,Mazda,Легковые автомобили,Слева,Коричневый,9860,Не инвалид,32,12,Павлодарская область
5279,50,32,4,Other,Renault,Легковые автомобили,Слева,Прочие,14079,Не инвалид,31,12,Восточно-Казахстанская область
3412,61,6,3,Other,Mercedes-Benz,Легковые автомобили,Слева,Белый,5294,Не инвалид,36,6,Северо-Казахстанская область
8238,43,14,7,Passat,Volkswagen,Легковые автомобили,Слева,Прочие,8548,Не инвалид,30,12,Северо-Казахстанская область
7917,34,16,8,Avensis,Toyota,Легковые автомобили,Слева,Белый,11342,Не инвалид,15,12,Астана


In [62]:
df.isnull().sum()

Age                        0
Driving_experience         0
Bonus_Malus                0
Model                      0
Brand                      0
Vehicle_type               0
Steering_wheel_location    0
Color                      0
Insurance_premium          0
Privileges                 0
Car_age                    0
Insurance_months           0
Region                     0
dtype: int64

In [64]:
df.describe()

,Age,Driving_experience,Bonus_Malus,Insurance_premium,Car_age,Insurance_months
count,10902.000000,10902.000000,10902.00000,10902.000000,10902.000000,10902.000000
mean,47.351862,17.956797,7.36718,10933.732618,25.777472,10.673271
std,12.795866,8.712328,1.70596,5732.619701,7.735468,2.523356
min,24.000000,0.000000,1.00000,66.000000,12.000000,1.000000
25%,37.000000,12.000000,7.00000,6412.750000,19.000000,11.000000
50%,46.000000,18.000000,8.00000,9479.500000,26.000000,12.000000
75%,56.000000,20.000000,9.00000,15667.000000,31.000000,12.000000
max,91.000000,64.000000,9.00000,59669.000000,70.000000,12.000000
